In [1]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"


/Users/csuarezgurruchaga/Desktop/Digital-House/CLASE_23/dsad_2021/common
default checking
Running command `conda list`... ok
jupyterlab=2.2.6 already installed
pandas=1.1.5 already installed
bokeh=2.2.3 already installed
seaborn=0.11.0 already installed
matplotlib=3.3.2 already installed
ipywidgets=7.5.1 already installed
pytest=6.2.1 already installed
chardet=4.0.0 already installed
psutil=5.7.2 already installed
scipy=1.5.2 already installed
statsmodels=0.12.1 already installed
scikit-learn=0.23.2 already installed
xlrd=2.0.1 already installed
Running command `conda install --yes nltk=3.5.0`... ok
Solving environment: ...working... done

# All requested packages already installed.


unidecode=1.1.1 already installed
pydotplus=2.0.2 already installed
pandas-datareader=0.9.0 already installed
flask=1.1.2 already installed


[<img src="https://www.digitalhouse.com/ar/logo-DH.png" width="400" height="200" align='right'>](http://digitalhouse.com.ar/)

# Práctica Guiada
## Regularización, Overfitting  y _Cross validation_

Analicemos nuevamente el dataset inmobiliario de Boston y apliquemos cross-validation. Hay un excelente [ejemplo](http://scikit-learn.org/stable/auto_examples/plot_cv_predict.html) en la página de scikit-learn. Usemos ese código y modifiquémoslo para comparar el ajuste con _cross-validation_ contra el caso sin _cross-validation_. Vas a necesitar [esta función](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html) para ver las métricas del modelo con _cross-validation_ .

Trabajar sobre el [ejemplo](http://scikit-learn.org/stable/auto_examples/plot_cv_predict.html) de cross-validation, agregando calculos de $r^2$. ¿Difiere el score usando CV del score obtenido sobre el set completos de datos? ¿En qué sentido? ¿Por qué?

In [2]:
import random
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import datasets, linear_model
from sklearn.model_selection import cross_val_score, train_test_split, KFold
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
  
random.seed(3)
boston = datasets.load_boston()
y = boston.target
X = pd.DataFrame(boston.data, columns=boston.feature_names)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)
lr = linear_model.LinearRegression()
cv = KFold(5, shuffle=True)
scores = cross_val_score(lr, X_train, y_train, cv=cv, scoring='r2')
scores, scores.mean()

(array([0.66861075, 0.67218542, 0.67382079, 0.77514668, 0.73160649]),
 0.7042740230783744)

## Regularización: Lasso

Lasso (en inglés: _least absolute shrinkage and selection operator_) es otra forma de regularization. A esta altura, deberías poder modificar los ejemplos del notebook aplicando el modelo [Lasso](http://scikit-learn.org/stable/modules/linear_model.html#lasso) de scikit-learn y su versión con _cross-validation_ `LassoCV`. 

La principal diferencia entre Lasso y Ridge es cómo se penaliza. Analizando la documentacion, buscá las diferencias conceptuales y prácticas entre ambos.

¿Cuántos coeficientes son exactamente cero?

¿Es el $\alpha$ óptimo seleccionado por CV el mismo sin importar la escala de las variables?

Recuerde que en Lasso y Ridge es necesario que lo features esten en la misma escala

In [3]:
model = linear_model.LassoCV().fit(X_train, y_train)
scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='r2')
dict(alpha=model.alpha_, scores=scores, mean_score=scores.mean(), zero_coefs=(model.coef_ == 0).sum())

{'alpha': 0.7051444556162021,
 'scores': array([0.6940235 , 0.75414865, 0.73775976, 0.53855066, 0.68009309]),
 'mean_score': 0.680915133487735,
 'zero_coefs': 3}

In [4]:
model = linear_model.LassoCV(normalize=True).fit(X, y)
scores = cross_val_score(model, X, y, cv=cv, scoring='r2')
dict(alpha=model.alpha_, scores=scores, mean_score=scores.mean(), zero_coefs=(model.coef_ == 0).sum())

{'alpha': 0.008002884283707804,
 'scores': array([0.66046858, 0.61037712, 0.69208088, 0.7285869 , 0.79099912]),
 'mean_score': 0.6965025188235014,
 'zero_coefs': 2}

In [7]:
model = linear_model.RidgeCV(normalize=True).fit(X, y)
scores = cross_val_score(model, X, y, cv=cv, scoring='r2')
dict(alpha=model.alpha_, scores=scores, mean_score=scores.mean(),zero_coefs=(model.coef_ == 0).sum())

{'alpha': 0.1,
 'scores': array([0.78413514, 0.6643472 , 0.70319262, 0.68352827, 0.68305048]),
 'mean_score': 0.703650742027053,
 'zero_coefs': 0}

La principal diferencia entre Lasso y Ridge es cómo se penaliza. Analizando la documentacion, buscá las diferencias conceptuales y prácticas entre ambos.

La regresion Lasso, utiliza la suma de los valores absolutos de los coeficientes beta de las features, sin incluir el intercepto. A su vez, con valores altos de ALPHA, es posible que penalice tanto a algunos coeficientes beta que los reduzca a cero, ya que no los considere necesario para predecir la variable target. Por otro lado tenemos la Regression Ridge, en la cual nuestro coeficiente de penalizacion ALPHA, multiplica a la suma de los cuadrados de los coeficientes beta del modelo, sin incluir al intercepto.

¿Cuántos coeficientes son exactamente cero?




La regression Lasso sin escalar, llevo a cero 3 Features, luego, al normalizarla, llevo 2 Features a cero.

¿Es el  𝛼  óptimo seleccionado por CV el mismo sin importar la escala de las variables?



No, el valor de alpha se ve afectado, es por esto que siempre al hacer Regularizaciones, debemos escalar primero las features

Recuerde que en Lasso y Ridge es necesario que lo features esten en la misma escala